In [20]:
!pip install --user rake-nltk
!pip install --user python-rake

from rake_nltk import Metric, Rake
import RAKE
import pandas as pd
import os
import string

OUTPUT_FOLDER = '/projets/prevision/output/aggressive/'
#OUTPUT_FOLDER = '/users/tweetcontextualization/acouprie/ukenvironmental/outputs/'
# cmd and regex used: grep -Eo '[^.]* ([a-z]*|.)([t]|[T])error([a-z]*|.) [^.]*\.' /projets/prevision/datasets/ukenvironmental/all_sentences.json

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [21]:
def define_metric(metric):
    result = ""
    if(metric == "degree"):
        result = "Metric.WORD_DEGREE"
    elif(metric == "frequency"):
        result = "Metric.WORD_FREQUENCY"
    elif(metric == "degree_to_frequency"):
        result = "Metric.DEGREE_TO_FREQUENCY_RATIO"
    # default
    else:
        result = "Metric.DEGREE_TO_FREQUENCY_RATIO"
    return result

def rake_setup(file, length, metric):
    #ignore = ['\"', '\'', '://', '...']
    r = Rake(stopwords=RAKE.SmartStopList(), max_length=length, ranking_metric=eval(define_metric(metric)))#, punctuations=ignore, ranking_metric=Metric.WORD_FREQUENCY) # Uses stopwords for english from NLTK, and all punctuation characters.
    r.extract_keywords_from_text(file)
    result = r.get_ranked_phrases_with_scores() # To get keyword phrases ranked highest to lowest.
    return result
    
def rake(file, length, metric, output_file):
    # compute rake
    result = rake_setup(file, length, metric)
    # write to file
    write_rake_to_file(result, file, output_file)
    
def write_rake_to_file(result, input_file, output_file):
    extract = {}
    f = input_file.lower().replace(" ", "")
    for element in result:
        occurence = len(f.split(element[1].replace(" ", "")))-1
        extract[element] = occurence
    with open(output_file,"w") as output:
        output.write("keyword,ngram,freq,rake\n")
        for element in result:
            output.write("{},{},{},{}\n".format(
                element[1].translate(str.maketrans('', '', string.punctuation)),
                len(element[1].split()),
                extract[element],
                element[0])
            )

freq_files = []
def word_frequency(file, output_file):
    freq_files.append(output_file)
    r = Rake(stopwords=RAKE.SmartStopList())
    r.extract_keywords_from_text(file)
    freq = r.get_word_frequency_distribution()
    with open(output_file,"w") as output:
        output.write("keyword,freq\n")
        for element in sorted(freq, key=freq.get, reverse=True):
            output.write(
                "{},{}\n".format(
                    element.replace(',', '').replace('"', ''),
                    freq[element]
                )
            )

files_1_3_ngrams = []
files_unigram = []
def rake_extraction(input_file, output_name):
    metrics = ["degree", "frequency", "degree_to_frequency"]
    folder = OUTPUT_FOLDER + output_name + "/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    # word frequency
    word_frequency(input_file, folder + "frequency_" + output_name + ".csv")
    # 1-3 ngrams
    for i in range(0, len(metrics)):
        rake_output = folder + output_name + "_1-3ngrams_rake_" + metrics[i] + ".csv"
        rake(input_file, 3, metrics[i], rake_output)
        files_1_3_ngrams.append(rake_output)
    # unigrams
    rake_output = folder + output_name + "_1ngrams_rake_" + metrics[2] + ".csv"
    files_unigram.append(rake_output)
    rake(input_file, 1, metrics[2], rake_output)

In [22]:
df = pd.read_csv('/projets/prevision/datasets/aggressive/release-files/eng/trac2_eng_train.csv')
AGGRESSIVE_DATASET = '/projets/prevision/datasets/aggressive/'
aggressive_files = ["cag_aggressive", "full_aggressive", "nag_aggressive"]
# extract data from csv to files
for file in aggressive_files:
    path = AGGRESSIVE_DATASET + file + ".txt"
    if os.path.isfile(path):
        os.remove(path)

# write data on different files
full_file = open('/projets/prevision/datasets/aggressive/full_aggressive.txt',"a")
cag_file = open('/projets/prevision/datasets/aggressive/cag_aggressive.txt',"a")
nag_file = open('/projets/prevision/datasets/aggressive/nag_aggressive.txt',"a")

for i in range(0,len(df)):
    full_file.write(df['Text'].iloc[i])
    if(df['Sub-task A'].iloc[i] == 'CAG'):
        cag_file.write(df['Text'].iloc[i])
    elif(df['Sub-task A'].iloc[i] == 'NAG'):
        nag_file.write(df['Text'].iloc[i])

full_file.close()
cag_file.close()
nag_file.close()

In [23]:
folders = ["cag_aggressive", "full_aggressive", "nag_aggressive", "terror"]
RESOURCE_FOLDER = '/projets/prevision/datasets/aggressive/'
for folder in folders:
    with open(RESOURCE_FOLDER + folder + ".txt", 'r') as file:
        rake_extraction(file.read(), folder)

In [24]:
def define_min_lines(files):
    min_nb_lines = ['', 9999, 0]
    i = 0
    for file in files:
        with open(file, 'r') as f:
            lines = f.readlines()
            nb_lines = len(lines)
            if nb_lines < min_nb_lines[1]:
                min_nb_lines = [i, nb_lines, lines[0].count(',')]
            i = i + 1
    return min_nb_lines

def order_files(files):
    mirror_files = []
    for file in files:
        mirror_files.append(file[::-1])
    mirror_files = sorted(mirror_files)
    result = []
    for file in mirror_files:
        result.append(file[::-1])
    return result

def merge_files(input_files, output_file):
    files = order_files(input_files)
    # define the max size of the file according to the smallest file to merge
    smallest_file = define_min_lines(files)
    path = OUTPUT_FOLDER + output_file
    # delete output file if exist to overwrite
    if os.path.isfile(path):
        os.remove(path)
    with open(path, 'a') as f:
        # write header
        for i in range(0, len(files)):
            # get name from path
            f.write("{},".format(files[i].split('/')[-1].split('.')[0]))
            for j in range(0,smallest_file[2]):
                f.write(',')
        f.write(",\n")
        # first file
        with open(files[smallest_file[0]], 'r') as cf:
            or_content = cf.readlines()
        or_content = [x.strip() for x in or_content]
        # extend lines to add columns
        for i in range(0, len(files)):
            if i == smallest_file[0]:
                continue
            with open(files[i], 'r') as cf:
                content = cf.readlines()
            content = [x.strip() for x in content]
            for j in range(0, smallest_file[1]):
                if i > smallest_file[0]:
                    or_content[j] = or_content[j] + ',' + content[j]
                else:
                    or_content[j] = content[j] + ',' + or_content[j]
        for j in range(0, smallest_file[1]):
            f.write(or_content[j] + "\n")

merge_files(freq_files, "all_frequencies.csv")
merge_files(files_1_3_ngrams, "all_1_3_ngrams.csv")
merge_files(files_unigram, "all_unigrams.csv")